In [6]:
import numpy as np
import time
from numpy import ones,vstack
from numpy.linalg import lstsq

import math
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from random import random, randint, uniform

import os
import subprocess
import shutil

from statistics import median

In [7]:
def parse_problem(problem_file):

    x1_obs = []
    x4_obs = []
    x7_obs = []
    t_obs = []

    with open(problem_file, "r") as f:
        for line in f.readlines():
            if "(time_obs" in line:
                t_obs += [float(line.strip().replace("(", "").replace(")", "").split(" ")[3])]
            elif "(x1_obs" in line:
                x1_obs += [float(line.strip().replace("(", "").replace(")", "").split(" ")[3])]
            elif "(x4_obs" in line:
                x4_obs += [float(line.strip().replace("(", "").replace(")", "").split(" ")[3])]
            elif "(x7_obs" in line:
                x7_obs += [float(line.strip().replace("(", "").replace(")", "").split(" ")[3])]


    observations = list(zip(x1_obs, x4_obs, x7_obs, t_obs))
    
    return observations

In [8]:
def generate_decoding_domain(observations, error=0.05, time_precision=0.05):
    
    pddl_str = "(define (domain platoon)\n"
    pddl_str += "\n"
    
    pddl_str += "(:requirements :fluents :typing :time)\n"
    pddl_str += "\n"
    
    # Predicates
    pddl_str += "(:predicates\n"
    pddl_str += "\t;HS locations\n"
    pddl_str += "\t(elf)\n"
    pddl_str += "\t(dreizehn)\n"
    pddl_str += "\n"    
    pddl_str += "\t;monitor locations\n"
    for i in range(len(observations)+1):
        pddl_str += "\t(monitor_{})\n".format(i)
    pddl_str += ")\n"
    pddl_str += "\n"
    
    
    # Functions
    pddl_str += "(:functions\n"
    pddl_str += "\t;HS variables\n"
    pddl_str += "\t(x1)\n"
    pddl_str += "\t(x2)\n"
    pddl_str += "\t(x3)\n"
    pddl_str += "\t(x4)\n"
    pddl_str += "\t(x5)\n"
    pddl_str += "\t(x6)\n"
    pddl_str += "\t(x7)\n"
    pddl_str += "\t(x8)\n"
    pddl_str += "\t(x9)\n"
    pddl_str += "\t(u)\n"
    pddl_str += "\n"
    
    pddl_str += "\t;monitor variables\n"
    pddl_str += "\t(running_time)  ;; time\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    
    # HS Continuous transitions
    
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += ";;; HS Continuous transitions\n"
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += "\n"
    
    pddl_str += "(:process flow-elf\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(elf)\n"
    pddl_str += "\t\t(>= (x1) -50)\n"
    pddl_str += "\t\t(>= (x4) -50)\n"
    pddl_str += "\t\t(>= (x7) -50)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(increase (x1) (* #t (x2)))\n"
    pddl_str += "\t\t(increase (x2) (* #t (+ (* (x3) -1) (u))))\n"
    pddl_str += "\t\t(increase (x3) (* #t (+ (* 1.605 (x1)) (+ (* 4.868 (x2)) (+ (* -3.5754 (x3)) (+ (* -0.8198 (x4)) (+ (* 0.427 (x5)) (+ (* -0.045 (x6)) (+ (* -0.1942 (x7)) (+ (* 0.3626 (x8)) (* -0.0946 (x9)))))))))) ))\n"
    pddl_str += "\t\t(increase (x4) (* #t (x5)))\n"
    pddl_str += "\t\t(increase (x5) (* #t (+ (x3) (* (x6) -1))))\n"
    pddl_str += "\t\t(increase (x6) (* #t (+ (* 0.8718 (x1)) (+ (* 3.814 (x2)) (+ (* -0.0754 (x3)) (+ (* 1.1936 (x4)) (+ (* 3.6258 (x5)) (+ (* -3.2396 (x6)) (+ (* -0.595 (x7)) (+ (* 0.1294 (x8)) (* -0.0796 (x9)))))))))) ))\n"
    pddl_str += "\t\t(increase (x7) (* #t (x8)))\n"
    pddl_str += "\t\t(increase (x8) (* #t (+ (x6) (* (x9) -1))))\n"
    pddl_str += "\t\t(increase (x9) (* #t (+ (* 0.7132 (x1)) (+ (* 3.573 (x2)) (+ (* -0.0964 (x3)) (+ (* 0.8472 (x4)) (+ (* 3.2568 (x5)) (+ (* -0.0876 (x6)) (+ (* 1.2726 (x7)) (+ (* 3.072 (x8)) (* -3.1356 (x9)))))))))) ))\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    
    pddl_str += "(:process flow-dreizehn\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(dreizehn)\n"
    pddl_str += "\t\t(>= (x1) -50)\n"
    pddl_str += "\t\t(>= (x4) -50)\n"
    pddl_str += "\t\t(>= (x7) -50)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(increase (x1) (* #t (x2)))\n"
    pddl_str += "\t\t(increase (x2) (* #t (+ (* (x3) -1) (u))))\n"
    pddl_str += "\t\t(increase (x3) (* #t (+ (* 1.605 (x1)) (+ (* 4.868 (x2)) (* -3.5754 (x3)))) ))\n"
    pddl_str += "\t\t(increase (x4) (* #t (x5)))\n"
    pddl_str += "\t\t(increase (x5) (* #t (+ (x3) (* (x6) -1))))\n"
    pddl_str += "\t\t(increase (x6) (* #t (+ (* 1.1936 (x4)) (+ (* 3.6258 (x5)) (* -3.2396 (x6)))) ))\n"
    pddl_str += "\t\t(increase (x7) (* #t (x8)))\n"
    pddl_str += "\t\t(increase (x8) (* #t (+ (x6) (* (x9) -1))))\n"
    pddl_str += "\t\t(increase (x9) (* #t (+ (* 0.7132 (x1)) (+ (* 3.573 (x2)) (+ (* -0.0964 (x3)) (+ (* 0.8472 (x4)) (+ (* 3.2568 (x5)) (+ (* -0.0876 (x6)) (+ (* 1.2726 (x7)) (+ (* 3.072 (x8)) (* -3.1356 (x9)))))))))) ))\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    
    # HS Discrete transitions
    
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += ";;; HS Discrete transitions\n"
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += "\n"
    
    pddl_str += "(:action disconnect\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(elf)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(not (elf))\n"
    pddl_str += "\t\t(dreizehn)\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    pddl_str += "(:action reconnect\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(dreizehn)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(not (dreizehn))\n"
    pddl_str += "\t\t(elf)\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"

    
    
    # Monitor Continuous transitions
    
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += ";;; Monitor Continuous transitions\n"
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += "\n"
    
    
    for i in range(len(observations)+1):
        pddl_str += "(:process flow-monitor_{}\n".format(i)
        pddl_str += "\t:parameters ()\n"
        pddl_str += "\t:precondition (and\n"
        pddl_str += "\t\t(monitor_{})\n".format(i)
        pddl_str += "\t)\n"
        pddl_str += "\t:effect (and\n"
        pddl_str += "\t\t(increase (running_time) (* #t 1.0))\n"
        pddl_str += "\t)\n"
        pddl_str += ")\n"
        pddl_str += "\n"
        
        
    # Monitor Discrete transitions
    
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += ";;; Monitor Discrete transitions\n"
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += "\n"
        
    for i in range(len(observations)):
        obs = observations[i]
        pddl_str += "(:event validate_{}\n".format(i+1)
        pddl_str += "\t:parameters ()\n"
        pddl_str += "\t:precondition (and\n"
        pddl_str += "\t\t(monitor_{})\n".format(i)
        pddl_str += "\t\t(<= {} (running_time)) ; t_{} - 0.05\n".format(obs[3] - time_precision, i+1)
        pddl_str += "\t\t(> {} (running_time)) ; t_{} + 0.05\n".format(obs[3] + time_precision, i+1)
        pddl_str += "\t\t(<= {} (x1)) ; \phi_{}\n".format(round(obs[0] - error,3), i+1)
        pddl_str += "\t\t(>= {} (x1)) ; \phi_{}\n".format(round(obs[0] + error,3), i+1)
        pddl_str += "\t\t(<= {} (x4)) ; \phi_{}\n".format(round(obs[1] - error,3), i+1)
        pddl_str += "\t\t(>= {} (x4)) ; \phi_{}\n".format(round(obs[1] + error,3), i+1)
        pddl_str += "\t\t(<= {} (x7)) ; \phi_{}\n".format(round(obs[2] - error,3), i+1)
        pddl_str += "\t\t(>= {} (x7)) ; \phi_{}\n".format(round(obs[2] + error,3), i+1)
        pddl_str += "\t)\n"
        pddl_str += "\t:effect (and\n"
        pddl_str += "\t\t(not (monitor_{}))\n".format(i)
        pddl_str += "\t\t(monitor_{})\n".format(i+1)
        pddl_str += "\t)\n"
        pddl_str += ")\n"
        pddl_str += "\n"
        
        
    pddl_str += ")"
    
    return pddl_str
    

In [9]:
def generate_decoding_problem(observations):
    
    pddl_str = "(define (problem problem_name) (:domain platoon)\n"
    pddl_str += "\n"

    # Initial state
    pddl_str += "(:init\n"
    pddl_str += "\t; HS initial state\n"
    pddl_str += "\t(= (x1) 0)\n"
    pddl_str += "\t(= (x2) 0)\n"
    pddl_str += "\t(= (x3) 0)\n"
    pddl_str += "\t(= (x4) 0)\n"
    pddl_str += "\t(= (x5) 0)\n"
    pddl_str += "\t(= (x6) 0)\n"
    pddl_str += "\t(= (x7) 0)\n"
    pddl_str += "\t(= (x8) 0)\n"
    pddl_str += "\t(= (x9) 0)\n"
    pddl_str += "\t(= (u) -4)\n"
    pddl_str += "\t(elf)\n"
    pddl_str += "\t; monitor initial state\n"
    pddl_str += "\t(= (running_time) 0)\n"
    pddl_str += "\t(monitor_0)\n"
    pddl_str += "\n"
    pddl_str += ")\n"
    
    pddl_str += "(:goal\n"
    pddl_str += "\t(and\n"
    pddl_str += "\t\t(monitor_{})\n".format(len(observations))
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += ")\n"
    
    return pddl_str
    

In [10]:
num_instances = 10
horizons = [2, 4, 6, 8, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

dfolder = "benchmarks/platoon/UPM_e"
if os.path.exists(dfolder):
    shutil.rmtree(dfolder)
os.makedirs(dfolder)


for pnum in range(num_instances):
    for h in horizons:
        dname = 'domain_{pnum}_{h}.pddl'.format(pnum=pnum, h=h)
        pname = 'inst_{pnum}_{h}.pddl'.format(pnum=pnum, h=h)
        
        observations = parse_problem("benchmarks/platoon/experiment/inst_{}_{}_5_100.pddl".format(pnum, h))
        observations = observations[1:]
        
        
        domain_str = generate_decoding_domain(observations)
        with open(dfolder + '/' + dname, 'w') as f:
            f.write(domain_str)
            
        problem_str = generate_decoding_problem(observations)
        with open(dfolder + '/' + pname, 'w') as f:
            f.write(problem_str)

In [11]:
#horizons = [2, 4, 6, 8, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
#dfolder = "benchmarks/platoon/HSE_Pe_std"

times = {}
instances = list(range(num_instances))

cmd0 = "cd {dfolder}".format(dfolder=dfolder)
os.system(cmd0)

time_budget = 300

for h in horizons:
    h_times = []
    working_instances = []
    print(instances)
    for pnum in instances:  
        
        try:
            dname = 'domain_{pnum}_{h}'.format(pnum=pnum, h=h)
            pname = 'inst_{pnum}_{h}'.format(pnum=pnum, h=h)

            cmd1 = "cd {dfolder}; upmc {dname}.pddl {pname}.pddl --custom 0.01 10 4 --clean --force".format(dfolder=dfolder, dname=dname, pname=pname)
            
            t = time.time()
            os.system(cmd1)
            elapsed = time.time() - t
            
            cmd2 = "cd {dfolder}; timeout {timeout} ./{dname}_planner -m 8000".format(dfolder=dfolder, dname=dname, timeout=int(300-elapsed))
            
            os.system(cmd2)
            elapsed = time.time() - t
            
            os.system("cd {dfolder}; rm *.transitions *.properties *.startstates".format(dfolder=dfolder))
            
            with open ("{dfolder}/{pname}_plan.pddl".format(dfolder=dfolder, pname=pname), "r") as f:
                content = f.read()
            assert content != ""

            #shutil.move(dfolder + "/{pname}_search_WAStar_h_hmrp_break_ties_arbitrary.npt".format(pname=pname), dfolder + "/solution_{pnum}_{h}.npt".format(pnum=pnum, h=h))
            h_times += [elapsed]
            working_instances += [pnum]
        except:
            print("instance {} failed".format(pnum))
        
    times[h] = h_times
    print(h, h_times)
    
    instances = working_instances
    
with open("{}/upm_e_001".format(dfolder), "w") as f:
    for k,h_times in times.items():
        f.write(" ".join(map(str, h_times)) + "\n")


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Call with the -l flag or read the license file for terms
and conditions of use.
Run this program with "-h" for the list of options.
Bugs, questions, and comments should be directed to
"giuseppe.dellapenna@univaq.it".

UPMurphi compiler last compiled date: Nov 15 2022


 Start UPMurphi translation... 

 ----- CONFIG FILE ----- 
PDDL Domain File: domain_0_2.pddl
PDDL Problem File: inst_0_2.pddl
The output model will be written on file: domain_0_2.m
The output external function file will be written on file: domain_0_2.h
 ----- DOMAIN SETTINGS ----- 
 Time discretisation: 0.010000
 Real number: (Integer): 10.000000 digits for the integer part
 Real number: (Fractional) 4.000000 digits for the fractional part


 ...Translation completed 


Errors: 0, warnings: 0

Universal Planner Murphi Release 3.0
Universal Planner for PDDL+ models with continous processes and events.

Universal Planner Murphi Release 3.0 :
Copyright (C) 2007 - 2015: G. Della Penna, B. Intri

[0:0:24.58] states explored: 2200000, actions fired: 2788645
  BFS level: 31, states queued: 805677, goals found: 0, invalid states: 0, max plan length: 0.31
  89503.66 states/sec, 113451.79 actions/sec, 0.66% memory used
[0:0:25.56] states explored: 2300000, actions fired: 2914596
  BFS level: 31, states queued: 842702, goals found: 0, invalid states: 0, max plan length: 0.31
  89984.35 states/sec, 114029.58 actions/sec, 0.69% memory used
[0:0:26.55] states explored: 2400000, actions fired: 3041404
  BFS level: 31, states queued: 879298, goals found: 0, invalid states: 0, max plan length: 0.31
  90395.48 states/sec, 114553.82 actions/sec, 0.72% memory used
[0:0:27.54] states explored: 2500000, actions fired: 3167380
  BFS level: 31, states queued: 916310, goals found: 0, invalid states: 0, max plan length: 0.31
  90777.05 states/sec, 115010.17 actions/sec, 0.75% memory used
[0:0:28.54] states explored: 2600000, actions fired: 3294757
  BFS level: 31, states queued: 952621, goals found

[0:1:1.13] states explored: 5900000, actions fired: 7461109
  BFS level: 33, states queued: 2169445, goals found: 0, invalid states: 0, max plan length: 0.33
  96515.62 states/sec, 122053.15 actions/sec, 1.76% memory used
[0:1:2.12] states explored: 6000000, actions fired: 7588148
  BFS level: 33, states queued: 2205926, goals found: 0, invalid states: 0, max plan length: 0.33
  96587.25 states/sec, 122153.06 actions/sec, 1.79% memory used
[0:1:3.11] states explored: 6100000, actions fired: 7714561
  BFS level: 33, states queued: 2242719, goals found: 0, invalid states: 0, max plan length: 0.33
  96656.63 states/sec, 122239.91 actions/sec, 1.82% memory used
[0:1:4.09] states explored: 6200000, actions fired: 7840175
  BFS level: 33, states queued: 2279912, goals found: 0, invalid states: 0, max plan length: 0.33
  96738.96 states/sec, 122330.71 actions/sec, 1.85% memory used
[0:1:5.08] states explored: 6300000, actions fired: 7967652
  BFS level: 33, states queued: 2316174, goals found

[0:1:37.72] states explored: 9600000, actions fired: 12139491
  BFS level: 34, states queued: 3530254, goals found: 0, invalid states: 0, max plan length: 0.34
  98239.87 states/sec, 124227.29 actions/sec, 2.86% memory used
[0:1:38.70] states explored: 9700000, actions fired: 12264948
  BFS level: 34, states queued: 3567526, goals found: 0, invalid states: 0, max plan length: 0.34
  98277.61 states/sec, 124264.92 actions/sec, 2.89% memory used
[0:1:39.69] states explored: 9800000, actions fired: 12391027
  BFS level: 34, states queued: 3604486, goals found: 0, invalid states: 0, max plan length: 0.34
  98304.74 states/sec, 124295.59 actions/sec, 2.92% memory used
[0:1:40.70] states explored: 9900000, actions fired: 12518309
  BFS level: 34, states queued: 3640845, goals found: 0, invalid states: 0, max plan length: 0.34
  98311.82 states/sec, 124312.90 actions/sec, 2.95% memory used
[0:1:41.70] states explored: 10000000, actions fired: 12645846
  BFS level: 34, states queued: 3677077, 

[0:2:14.79] states explored: 13300000, actions fired: 16817073
  BFS level: 35, states queued: 4891463, goals found: 0, invalid states: 0, max plan length: 0.35
  98672.01 states/sec, 124764.99 actions/sec, 3.96% memory used
[0:2:15.78] states explored: 13400000, actions fired: 16942782
  BFS level: 35, states queued: 4928609, goals found: 0, invalid states: 0, max plan length: 0.35
  98689.06 states/sec, 124781.13 actions/sec, 3.99% memory used
[0:2:16.78] states explored: 13500000, actions fired: 17069924
  BFS level: 35, states queued: 4965038, goals found: 0, invalid states: 0, max plan length: 0.35
  98698.64 states/sec, 124798.39 actions/sec, 4.02% memory used
[0:2:17.79] states explored: 13600000, actions fired: 17197787
  BFS level: 35, states queued: 5001106, goals found: 0, invalid states: 0, max plan length: 0.35
  98700.92 states/sec, 124811.58 actions/sec, 4.05% memory used
[0:2:18.80] states explored: 13700000, actions fired: 17324470
  BFS level: 35, states queued: 50377

...

[0:4:5.81] states explored: 24400000, actions fired: 30887840
  BFS level: 36, states queued: 8956080, goals found: 0, invalid states: 0, max plan length: 0.36
  99263.66 states/sec, 125657.38 actions/sec, 7.27% memory used
[0:4:6.81] states explored: 24500000, actions fired: 31014547
  BFS level: 36, states queued: 8992726, goals found: 0, invalid states: 0, max plan length: 0.36
  99266.64 states/sec, 125661.63 actions/sec, 7.30% memory used
[0:4:7.81] states explored: 24600000, actions fired: 31142350
  BFS level: 36, states queued: 9028825, goals found: 0, invalid states: 0, max plan length: 0.36
  99269.60 states/sec, 125670.27 actions/sec, 7.33% memory used
[0:4:8.81] states explored: 24700000, actions fired: 31269841
  BFS level: 36, states queued: 9065079, goals found: 0, invalid states: 0, max plan length: 0.36
  99272.54 states/sec, 125677.59 actions/sec, 7.36% memory used
[0:4:9.82] states explored: 24800000, actions fired: 31398402
  BFS level: 36, states queued: 9100799, g

[0:4:42.94] states explored: 28100000, actions fired: 35613400
  BFS level: 36, states queued: 10293300, goals found: 0, invalid states: 0, max plan length: 0.36
  99314.34 states/sec, 125869.09 actions/sec, 8.37% memory used
[0:4:43.95] states explored: 28200000, actions fired: 35741751
  BFS level: 36, states queued: 10329124, goals found: 0, invalid states: 0, max plan length: 0.36
  99313.26 states/sec, 125873.40 actions/sec, 8.40% memory used
[0:4:44.97] states explored: 28300000, actions fired: 35871747
  BFS level: 36, states queued: 10364126, goals found: 0, invalid states: 0, max plan length: 0.36
  99308.70 states/sec, 125879.03 actions/sec, 8.43% memory used
[0:4:45.99] states explored: 28400000, actions fired: 36002103
  BFS level: 36, states queued: 10398948, goals found: 0, invalid states: 0, max plan length: 0.36
  99304.17 states/sec, 125885.88 actions/sec, 8.46% memory used
[0:4:47.02] states explored: 28500000, actions fired: 36131995
  BFS level: 36, states queued: 1

rm: cannot remove '*.transitions': No such file or directory
rm: cannot remove '*.properties': No such file or directory
rm: cannot remove '*.startstates': No such file or directory


instance 9 failed
2 []
[]
4 []
[]
6 []
[]
8 []
[]
10 []
[]
20 []
[]
30 []
[]
40 []
[]
50 []
[]
60 []
[]
70 []
[]
80 []
[]
90 []
[]
100 []


OSError: [Errno 28] No space left on device

In [7]:
for k,v in times.items():
    print(k, len(v), median(v))

StatisticsError: no median for empty data